In [13]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [15]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [18]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

In [21]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [22]:
input_shape = (256, 256, 3)

In [23]:
# Creating the base model
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)
base_model.trainable = False

In [31]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 131073    
                                                                 
Total params: 23718785 (90.48 MB)
Trainable params: 131073 (512.00 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [33]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [34]:
history = model.fit(x_train,epochs=5, batch_size=100, validation_data=x_test)

Epoch 1/5
89/89 [==============================] - 2539s 28s/step - loss: 1.2544 - accuracy: 0.5938 - val_loss: 0.7480 - val_accuracy: 0.5233
Epoch 2/5
89/89 [==============================] - 2517s 28s/step - loss: 0.5921 - accuracy: 0.6674 - val_loss: 0.6456 - val_accuracy: 0.6350
Epoch 3/5
89/89 [==============================] - 2437s 27s/step - loss: 0.5708 - accuracy: 0.6964 - val_loss: 0.9144 - val_accuracy: 0.5233
Epoch 4/5
89/89 [==============================] - 2507s 28s/step - loss: 0.5325 - accuracy: 0.7176 - val_loss: 0.5985 - val_accuracy: 0.6717
Epoch 5/5
89/89 [==============================] - 2530s 28s/step - loss: 0.5808 - accuracy: 0.7033 - val_loss: 0.7908 - val_accuracy: 0.6000
